In [158]:
import datetime
import pandas as pd
import numpy as np
import os
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

loc = os.getcwd()
sys.path.append(os.path.abspath('nytimesarticle-0.1.0'))

from get_NYT import get_NYT
'''
#Put into a nice, clean JSON in the gitignore directory
'''
ACCESS_KEY='18397b8a5f249028c58514c15cfa93fa'

from get_fx import get_fx
url = 'http://apilayer.net/api/historical?'

In [323]:
if __name__ == '__main__':
    
    today = datetime.date.today()
    today_dt = str(today.year)+str(today.month).zfill(2)+str(today.day).zfill(2)
    brexit_dt = datetime.date(2016,6,22)
    start = str(brexit_dt.year)+str(brexit_dt.month).zfill(2)+str(brexit_dt.day).zfill(2)
    hist = today - brexit_dt

    '''Load price change file
    '''
    df_chg = pd.read_csv('GBPUSD_CHNG.csv',header=None,names=['date','change'])
    df_chg = df_chg.set_index(pd.DatetimeIndex(df_chg['date']))

    '''Grab new FX prices and calculate changes
    '''
    max_dt = max(pd.to_datetime(df_chg['date']))
    begin = max_dt.date() + datetime.timedelta(1)
    if max_dt.date() < today:
        fx_price_gap = get_fx(ACCESS_KEY, url, begin, today)
        dt_index = pd.date_range(max_dt,periods=(pd.to_datetime(today)-max_dt).days+1,freq='D')[::-1][:-1]
        dt_gap = [d.strftime('%m/%d/%y') for d in dt_index]
        dt_gap = pd.DataFrame(dt_gap)
        fx_prices_more = pd.DataFrame(fx_price_gap)
        fx_px_delta = pd.concat([dt_gap,fx_prices_more], axis=1)
        fx_px_delta.columns = ['date','price']
        fx_price_hist = pd.read_csv('GBPUSD.csv',names=['date','price'],header=0)
        fx_prices = pd.concat([fx_px_delta,fx_price_hist],axis=0)
        fx_prices.to_csv('GBPUSD.csv',index=False)
        fx_prices = pd.read_csv('GBPUSD.csv',names=['date','price'],header=0)

        '''Calculate 1-day price change directions
        '''
        df_move = np.array(fx_prices.iloc[:-1,1]) - np.array(fx_prices.iloc[1:,1])
        df_signal = np.sign(df_move)
        df_chg = pd.Series(df_signal,index=fx_prices.iloc[:-1,0])
        df_chg.to_csv('GBPUSD_CHNG.csv')

In [353]:
    '''Backfill NYT articles'''
    df_NYT = pd.read_csv('NYTimes_20160623-20161128.csv',header=0,names=['dates','content'])
    df_NYT.set_index('dates',inplace=True)
    max_NYT_dt = max(pd.to_datetime(df_NYT.index))
    if max_NYT_dt.date() < today:
        begin_NYT = max_NYT_dt.date() + datetime.timedelta(1)
        begin_dt = str(begin_NYT.year)+str(begin_NYT.month).zfill(2)+str(begin_NYT.day).zfill(2)
        dts_NYT_fill, corpus_NYT_fill = get_NYT(begin_dt, today_dt)
        datetimes = [pd.to_datetime(dt) for dt in dts_NYT_fill]
        max_dt = max(datetimes)
        curr_articles = sum(np.array(datetimes) == max_dt)
        print "number of articles today: {}".format(curr_articles)
    #dt_NYT_gap = [d.strftime('%m/%d/%y') for d in dts_NYT_fill]
    dt_NYT_gap = pd.DataFrame(dts_NYT_fill)
    df_NYT_corpus = pd.DataFrame(corpus_NYT_fill)
    pd_NYT_fill = pd.concat([dt_NYT_gap,df_NYT_corpus], axis=1)
    pd_NYT_fill.columns=['date','content']
    pd_NYT_fill.set_index('date',inplace=True)
    pd_NYT_fill

5
number of articles today: 3


,content
date,
2016-11-29,LONDON It s official or is it The B...
2016-11-30,The in s center right presidential primary...
2016-12-02,LONDON An election here was supposed to b...
2016-12-02,TURIN Italy s prime minister Matteo R...
2016-12-02,LONDON In a surprise upset a candidate of...


In [472]:
    #pd_NYT_fill.to_csv('NYTimes_20160623-20161202.csv',header='True')
    df_NYT = pd.read_csv('NYTimes_20160623-20161202.csv',header=0,names=['dates','content'])
    df_NYT.set_index('dates',inplace=True)
    end = '20161202'
    end_dt = datetime.date(int(end[:4]),int(end[4:6]),int(end[-2:]))

In [473]:
    ind_test = str(end_dt.month)+'/'+str(end_dt.day)+'/'+str(end_dt.year-2000)

In [517]:
    corpus = df_NYT.loc[:ind_test]['content']
    mask = corpus.index.isin([ind_test])
    corpus_train = corpus.loc[~mask]
    X_train = count_vec.fit_transform(corpus_train)

In [523]:
    Y = []
    for dt in corpus.index:
        Y.append(df_chg.loc[dt])
    df_px_article = pd.DataFrame(Y,index=pd.DatetimeIndex(corpus.index),columns=['PriceChg'])

In [535]:
    corpus_test = df_NYT.loc[ind_test]['content']
    #X_train = X_train[:-len(X_test.index)]
    Y_train = Y[:-len(corpus_test.index)]
    Y_test = Y[-len(corpus_test):]

In [536]:
    if isinstance(corpus_test, str):
        corpus_test = [corpus_test]
    else: 
        corpus_test = [d for d in corpus_test]
    X_test = count_vec.transform(corpus_test)
    Y_test = [df_chg[ind_test] for i in X_test]

In [537]:
print len(Y_train)
print X_train.shape

48
(48, 6603)


In [538]:
print len(Y_test)
print X_test.shape

3
(3, 6603)


In [539]:
    clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
    clf.fit(X_train, Y_train)
    print clf.predict(X_test)

[-1. -1. -1.]


In [540]:
    X_tfidf_train = tfidf_vec.fit_transform(corpus_train)
    X_tfidf_test = tfidf_vec.transform(corpus_test)
    clf.fit(X_tfidf_train, Y_train)
    print clf.predict(X_tfidf_test)

[-1. -1. -1.]


In [304]:
        #df_chg.set_index('date',inplace=True)
        #df_chg.set_index(pd.to_datetime(df_chg['date']),inplace=True)
        start = '20161130'
        end = '20161130'
        end_dt = datetime.date(int(end[:4]),int(end[4:6]),int(end[-2:]))
        '''
        Get NYTimes articles
        NEED TO FIX.
        '''
        dates, corpus = get_NYT(start, end)

        datetimes = [pd.to_datetime(dt) for dt in dates]
        max_dt = max(datetimes)
        curr_articles = sum(np.array(datetimes) == max_dt)
        print "number of articles today: {}".format(curr_articles)

1
number of articles today: 1


In [177]:
pd_corpus_today = pd.DataFrame(corpus,index=dates)

In [143]:
NYT_dates = [d for d in df_NYT.index]

In [180]:
df_NYT_new = pd.concat([df_NYT,pd_corpus_today], axis=0)
df_NYT_new
df_NYT_new.to_csv('NYTimes_20160623-20161128.csv',header='True')

In [185]:
df_NYT_new = pd.read_csv('NYTimes_20160623-20161128.csv',header=0,names=['dates','content'])
df_NYT_new.tail()

,dates,content
41,11/18/16,PARIS In the age of Donald J Trump Br...
42,11/22/16,LONDON Once again President elect seems ...
43,11/22/16,LONDON When President elect in recent da...
44,11/23/16,LONDON After years of tough spending curbs...
45,11/28/16,JERUSALEM Within days of s election to ...


In [203]:
dates = df_NYT_new['dates']
dates

0      6/23/16
1      6/23/16
2      6/23/16
3      6/23/16
4      6/23/16
5      6/27/16
6      6/27/16
7      6/27/16
8      6/27/16
9      6/27/16
10     6/27/16
11     6/27/16
12     6/27/16
13     6/27/16
14     6/27/16
15     6/30/16
16     6/30/16
17     6/30/16
18     6/30/16
19     6/30/16
20     6/30/16
21      7/5/16
22      7/5/16
23      7/5/16
24      7/5/16
25      7/8/16
26      7/8/16
27      7/8/16
28      7/8/16
29      7/8/16
30      7/8/16
31     7/11/16
32     7/14/16
33     7/14/16
34     7/17/16
35     7/22/16
36     7/22/16
37     7/22/16
38     9/25/16
39     11/4/16
40     11/8/16
41    11/18/16
42    11/22/16
43    11/22/16
44    11/23/16
45    11/28/16
Name: dates, dtype: object

In [256]:
new_dates, new_corpus = get_NYT('20161129', '20161129')

1


In [239]:
df_chg = pd.read_csv('GBPUSD_CHNGv2.csv',names=['date','price'],header=0)
df_chg.set_index('date',inplace=True)
df_chg.tail()

,price
date,
6/27/16,-1
6/26/16,-1
6/25/16,0
6/24/16,-1
6/23/16,1


In [254]:
Y_train = Y

In [121]:
count_vec = CountVectorizer(min_df=1,stop_words='english',lowercase=True)
tfidf_vec = TfidfVectorizer(sublinear_tf=True, max_df=0.5,stop_words='english')

In [257]:
X_test = new_corpus
#corpus = df_NYT_new['content']

In [206]:
X_train = count_vec.fit_transform(corpus)

In [258]:
X_test = count_vec.transform(new_corpus)